In [ ]:
import pandas as pd


In [1]:
imUsingColab = False
shouldDownload = True # Set this to False after the first run to avoid redownloading the dataset
if shouldDownload:
  if imUsingColab:
    # Generate the .env file within the colab environment programatically
    file = open('.env','w') 
    file.write('KAGGLE_USERNAME=philli') # Populate this with your KAGGLE_USERNAME DO NOT COMMIT!!
    file.write('\nKAGGLE_KEY=e8ea661b0f70ffb758fc2da6a77b8b51') # Populate this with your KAGGLE api Key DO NOT COMMIT!!
    file.close()
    # Fetch the downloadData dependency 
    import urllib.request
    urllib.request.urlretrieve ("https://raw.githubusercontent.com/lihaojin/Movie_Data_Analysis/master/downloadData.py", "downloadData.py")

  import downloadData
  downloadData.download(imUsingColab)


Couldn't Find dataset downloading now
b'Downloading the-movies-dataset.zip to /Users/candicezhong/Documents/Github/Movie_Data_Analysis\n\n'
Opening zip file
['ratings.csv', 'ratings_small.csv', 'links.csv', 'links_small.csv', 'keywords.csv', 'movies_metadata.csv', 'credits.csv']
Loaded Zip File
Attempting extraction
Extraction Completed
